In [2]:
import inspect
import re

import getpass
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback

OPENAI_API_KEY = getpass.getpass("OpenAI API Key: ")


In [3]:
davinci = OpenAI(
    model_name="text-davinci-003",
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)

In [4]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        response = chain.run(query)
        print(f"Spent a total of {cb.total_tokens} tokens")
    
    return response

### Utility chains

In [5]:
davinci_math = LLMMathChain(llm=davinci, verbose=True)

In [6]:
query = "What is 13 raised to the .3234 power?"

response = count_tokens(davinci_math, query)

print(response)



> Entering new LLMMathChain chain...
What is 13 raised to the .3234 power?
```python
import math
print(math.pow(13, .3234))
```

Answer: 2.2921829624217414

> Finished chain.
Spent a total of 272 tokens
Answer: 2.2921829624217414



In [8]:
print(davinci_math.prompt.template)

You are GPT-3, and you can't do math.

You can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.

So we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:

Question: ${{Question with hard calculation.}}
```python
${{Code that prints what you need to know}}
```
```output
${{Output of your code}}
```
Answer: ${{Answer}}

Otherwise, use this simpler format:

Question: ${{Question without hard calculation}}
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?

```python
print(37593 * 67)
```
```output
2518731
```
Answer: 2518731

Question: {question}



In [12]:
standard_prompt = PromptTemplate(input_variables=["question"], template="{question}")

standard_davinci_chain = LLMChain(llm=davinci, prompt=standard_prompt)

query = "What is 13 raised to the .3234 power?"
response = count_tokens(standard_davinci_chain, query)

print(response)

Spent a total of 25 tokens


13^0.3234 ≈ 2.845


In [15]:
standard_prompt.format(question=query)

'What is 13 raised to the .3234 power?'

In [17]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [18]:
clean_extra_spaces_chain = TransformChain(
    transform=transform_func,
    input_variables=["text"],
    output_variables=["output_text"]
)

In [24]:
query= "A random text  with   some irregular spacing.\n\n\n     Another one   here as well."
print(clean_extra_spaces_chain.run({"text": query}))

# it also works in this way without the dictionary format
print(clean_extra_spaces_chain.run(query))

A random text with some irregular spacing.
 Another one here as well.
A random text with some irregular spacing.
 Another one here as well.


In [25]:
template = """Paraphrase this text:

{output_text}

In the style of {style}.

Paraphrase: """

prompt = PromptTemplate(input_variables=["output_text", "style"], template=template)

In [26]:
style_paraphrase_chain = LLMChain(llm=davinci, prompt=prompt, output_key="final_output")
